In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Dataroma

https://www.dataroma.com/m/holdings.php?m=psc

### Target

1. Track holding history automatically.

2. 

# Yahoo Finance

https://finance.yahoo.com/quote/CMG?p=CMG&.tsrc=fin-srch

### Target

1. Crawl the past price of the traget stock.

2. Combine the dataset sysmatically. -> Generate the right dataframe format as an output.

3. Compute the essential statistics with Python or R.

### Inset Keywords

In [2]:
keywords = [i for i in input().split()]

CMG HLT LOW QSR BRK-B SBUX HHC A


CMG HLT LOW QSR BRK-B SBUX HHC A

### Selenium

In [3]:
import requests
from bs4 import BeautifulSoup
import sys
import codecs
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import requests
import time
from fake_useragent import UserAgent
import csv
import pandas as pd

In [4]:
#Use fake UserAgent
def set_header_user_agent():
    user_agent = UserAgent()
    return user_agent.random

#open browser #打開瀏覽器，但不要載入圖片
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values': {
        'images': 2,
        #'javascript': 2
    }
}
options.add_experimental_option('prefs', prefs)

#ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
#headers = {'User-Agent': 'User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
#user_agent = set_header_user_agent()
#headers={ 'user-agent': user_agent }
#options.add_argument("user-agent={}".format(user_agent)) # 使用偽造的 user-agent
#options.add_argument(headers)

driver = webdriver.Chrome('./chromedriver', chrome_options=options)


#define urls
url = 'https://finance.yahoo.com/quote/'

#for loop
i = 1
for keyword in keywords:

    #hist url
    hist_url = url+keyword+"/history?p="+keyword

    #開啟網頁
    driver.get(hist_url)
    driver.implicitly_wait(10)

    #點日期
    driver.find_element_by_xpath("""//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/div/div/div/span""").click()
    #點max
    driver.find_element_by_xpath("""//*[@id="dropdown-menu"]/div/ul[2]/li[4]/button/span""").click()
    #點apply
    driver.find_element_by_xpath("""//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/button/span""").click()
    driver.implicitly_wait(20)

    #html parse 抓下載檔案的網址
    hist_soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = hist_soup.select(".Fl(end).Mt(3px).Cur(p)") #帶有空格的class需要用.來取

    #CSV file link
    csv_url = links[0]['href']
    df = pd.read_csv(csv_url)
    df = df[['Date','Close']]
    df.rename(columns={'Close':keyword}, inplace=True) #rename the column after the specific stock
    
    #assign不同的股票到不同的dataframe
    names = locals()
    names['df%s' % i] = df
    i+=1

df = df1 #df是最後要的pooled的dataFrame，先assign成df1，再逐個跟其後的合併
for j in range(2, i):
    df = pd.merge(df, names['df%s' % j], how = 'left', on = ["Date"])
    #check how to use pd.merge() in the ref: 
    #https://pandas-docs.github.io/pandas-docs-travis/user_guide/merging.html


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: use options instead of chrome_options


In [5]:
df.head()

,Date,CMG,HLT,LOW,QSR,BRK-B,SBUX,HHC,A
0,2006-01-26,44.000000,NaN,32.595001,NaN,58.820000,15.745,NaN,24.399141
1,2006-01-27,42.200001,NaN,32.555000,NaN,58.759998,15.975,NaN,24.420601
2,2006-01-30,44.799999,NaN,31.990000,NaN,58.560001,15.935,NaN,24.606581
3,2006-01-31,47.520000,NaN,31.775000,NaN,58.639999,15.850,NaN,24.256081
4,2006-02-01,46.560001,NaN,31.665001,NaN,58.299999,15.680,NaN,25.007153


In [7]:
df.tail()

,Date,CMG,HLT,LOW,QSR,BRK-B,SBUX,HHC,A
3580,2020-04-17,820.270020,75.620003,97.099998,45.980000,191.199997,77.099998,51.990002,79.550003
3581,2020-04-20,808.729980,72.279999,95.129997,44.080002,188.750000,75.320000,49.570000,77.370003
3582,2020-04-21,786.690002,71.349998,92.620003,42.369999,183.479996,72.720001,49.049999,73.750000
3583,2020-04-22,882.260010,70.790001,94.949997,45.250000,186.580002,77.449997,50.990002,75.169998
3584,2020-04-23,867.020020,71.940002,94.900002,46.180000,185.630005,75.150002,50.900002,75.089996


# Save Files

In [8]:
df.to_csv('output.csv', index=False)